# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd
import numpy as np

months ={
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9, 
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12,
        'January' : 1,
        'February' : 2,
        'March' : 3,
        'April' : 4,
        'May' : 5,
        'June' : 6,
        'July' : 7,
        'August' : 8,
        'September' : 9, 
        'October' : 10,
        'November' : 11,
        'December' : 12
}


doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [30]:
def date_sorter():
    
    # Your code here
    date1 = df.str.extractall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*[ -](\d{1,2})[a-z\.\,]*[ -](\d{4})')
    date1.columns = ["Month", "Day", "Year"]
    date1 = date1.reset_index()
    
    date2 = df.str.extractall(r'(0?[1-9]|1[0-2])[\/\-](0?[1-9]|[12]\d|30|31)[\/\-](\d{4}|\d{2})')
    date2.columns = ["Month", "Day", "Year"]
    date2[['Month', 'Year']] = date2[['Month', 'Year']].astype(int)
    date2 = date2.reset_index()
    
    date3 = df.str.extractall(r'(\d{1,2})?[ -]?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.\,]*[ -](\d{4})')
    date3.columns = ["Day", "Month", "Year"]
    date3 = date3.reset_index()
    
    ################################################### Only Month and Year Data ###############################
    date4 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    date4.insert(0, column='Day', value=np.nan)
    date4.columns = ["Day","Month", "Year"]
    date4[['Month', 'Year']] = date4[['Month', 'Year']].astype(int)
    date4['Year'] = date4['Year'].where(date4['Year']<2020)
    #date4['Year'] = date4['Year'].where(date4['Year']>1900)
    date4['Month'] = date4['Month'].where(date4['Month']<13)
    date4['Month'] = date4['Month'].where(date4['Month']>0)
    date4.dropna(subset=['Month'], how='all', inplace=True)
    date4[['Month', 'Year']] = date4[['Month', 'Year']].astype(int)
    #date4 = (date4[~date4.Year.isin(date2.Year)])
    date4 = date4.reset_index()
    
    ################################################## Only Year Data ########################################
    date5 = df.str.extractall(r'(\d{4})')
    date5.insert(0, column='Day', value=np.nan)
    date5.insert(1, column='Month', value=np.nan)
    date5.columns = ["Day","Month", "Year"]
    date5['Year'] = date5['Year'].astype(int)
    date5['Year'] = date5['Year'].where(date5['Year']<2020)
    #date5['Year'] = date5['Year'].where(date5['Year']>1900)
    date5.dropna(subset=['Year'], how='all', inplace=True)
    date5['Year'] = date5['Year'].astype(int)
    date5 = date5.reset_index()
    
    ############################################## Now the answer ##############################################
    
    
    output = date1.append(date2[~date2.level_0.isin(date1.level_0)])
    output = output.append(date3[~date3.level_0.isin(output.level_0)])
    output = output.append(date4[~date4.level_0.isin(output.level_0)])
    output = output.append(date5[~date5.level_0.isin(output.level_0)])
    output[['Year']] = output[['Year']].astype(int)
    output[['Year']] = output[['Year']].astype(str)
    output.Year = np.where(output.Year.apply(len)==2, "19"+output.Year, output.Year)
    output.Month = output.Month.replace(months)
    output = output.fillna('1')
    output[['Day', 'Month', 'Year']] = output[['Day', 'Month', 'Year']].astype(int)
    #output['date'] = output[['Year', 'Month', 'Day']].apply(lambda row: '-'.join(row.values.astype(str)), axis = 1)
    output["date"] = pd.to_datetime(output.loc[:,["Year", "Month", "Day"]])
    output = output.sort_values(['date', 'level_0']).reset_index(drop=True)
    dff = output.level_0

    dff = pd.Series(dff)
    return dff # Your answer here 
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64